We start with system commands to install the required library in the user space.

<span style="color:red"> If this kernel is hosted locally only run this once and restart the kernel afterwords! </span>

In [ ]:
import sys
!{sys.executable} -m pip install pandas --user
!{sys.executable} -m pip install matplotlib --user
!{sys.executable} -m pip install scipy --user
!{sys.executable} -m pip install bokeh --user

# <span style="color:green">Start Here</span>

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import collections
import numbers

#for if I end up using bokeh
#output_notebook()

Defines a data structure to hold all the relevant information for a option at a given time

In [2]:
class OptionData:
    def __init__(self,S,δ,r,σ,K,T):
        self.S = S
        self.δ = δ
        self.r = r
        self.σ = σ
        self.K = K
        self.T = T
        
        #renamed to fit how these formulas are normally expressed
        N = st.norm.cdf
        ϕ = st.norm.pdf
        
        d1 = (np.log(S/K)+(r-δ+0.5*np.power(σ,2))*T)/(σ*np.sqrt(T))
        d2 = (np.log(S/K)+(r-δ-0.5*np.power(σ,2))*T)/(σ*np.sqrt(T))
        self.d1 = d1
        self.d2 = d2
        
        self.C_Eur = S*np.exp(-δ*T)*N(d1)-K*np.exp(-r*T)*N(d2)
        self.P_Eur = K*np.exp(-r*T)*N(-d2)-S*np.exp(-δ*T)*N(-d1)
        
        #European greeks
        self.Δ_Call=np.exp(-δ*T)*N(d1)
        self.Δ_Put=-np.exp(-δ*T)*N(-d1)
        
        self.Γ_Call=(1.0/(S*self.σ*np.sqrt(T)))*np.exp(-δ*T)*ϕ(d1)
        self.Γ_Put=self.Γ_Call
        
        #Scaled by 1/100
        self.ν_Call=(S*np.exp(-δ*T)*ϕ(d1)*np.sqrt(T))/100.0
        self.ν_Put=self.ν_Call
        
        #Scaled by 1/365
        self.Θ_Call=(δ*S*np.exp(-δ*T)*N(d1)-r*K*np.exp(-r*T)*N(d2)-(σ/(2*np.sqrt(T)))*S*np.exp(-δ*T)*ϕ(d1))/365.0
        self.Θ_Put=(r*K*np.exp(-r*T)*N(-d2)-δ*S*np.exp(-δ*T)*N(-d1)-(self.σ/(2*np.sqrt(T)))*S*np.exp(-δ*T)*ϕ(d1))/365.0
        
        #Scaled 1/10,000
        self.ρ_Call=(T*K*np.exp(-r*T)*N(d2))/10000.0
        self.ρ_Put=(-T*K*np.exp(-r*T)*N(-d2))/10000.0
        
        #Scaled 1/10,000
        self.Ψ_Call=(-T*S*np.exp(-δ*T)*N(d1))/10000.0
        self.Ψ_Put=(T*S*np.exp(-δ*T)*N(-d1))/10000.0
        
        
def toFrame(obj):
    return pd.DataFrame.from_dict({k:v if hasattr(v, '__iter__') else [v] for k,v in obj.__dict__.items()})
        

In [6]:
ourData = OptionData(S = 68.04, K = 67.50, σ = 0.2433, r = 0.0180, T = 173/365, δ = 0.0118)

In [7]:
toFrame(ourData)

,S,δ,r,σ,K,T,d1,d2,C_Eur,P_Eur,...,Γ_Call,Γ_Put,ν_Call,ν_Put,Θ_Call,Θ_Put,ρ_Call,ρ_Put,Ψ_Call,Ψ_Put
0,68.04,0.0118,0.018,0.2433,67.5,0.473973,0.148865,-0.018636,4.868006,4.134056,...,0.034426,0.034426,0.183785,0.183785,-0.013326,-0.012213,0.001562,-0.00161,-0.001793,0.001414


# Monte Carlo Simulation

In [13]:
#normal and poission random variables
from scipy.stats import norm, poisson

#returns an iterator of trials allowing for laziness to discard uneeded data
def runMonteCarlo(iterations,trials,stockData,exerciseFun,antitheticVariate=True):
    r, δ, σ ,T = stockData.r, stockData.δ, stockData.σ ,stockData.T
    h = T/iterations
    rootH = np.sqrt(h)
    while trials>0:
        odd = True
        if antitheticVariate and not odd:
            norms = -norms
        else:
            norms = norm.rvs(size=iterations)
            trials -= 1
        S = stockData.S
        #These could be computed lazily as well, however we already allocated
        #an array for the random values that we can reuse
        prices = norms
        for i in range(0,iterations):
            Snext = S*np.exp((r - δ - 0.5*np.power(σ,2))*h + σ*rootH*norms[i])
            prices[i] = Snext
            S=Snext
        
        call,put = exerciseFun(prices,stockData)
        
        yield prices,call,put

# Exercise Functions

In [10]:
def europeanExerciseFunction(prices,stockData) :
    call = np.max(prices[-1]-stockData.K,0)
    put = np.max(stockData.K-prices[-1],0)
    return call,put
    
def asianArithmeticExerciseFunction(prices,stockData) :
    arithMean = np.mean(prices)
    call = max(arithMean - stockData.K,0)
    put = max(stockData.K - arithMean,0)
    return call,put

def asianGeometricExerciseFunction(prices,stockData) :
    geoMean = reduce(lambda k,p: np.power(p,1/len(prices))*k, prices, stockData.K)
    call = max(geomMean - stockData.K,0)
    put = max(stockData.K - geomMean,0)
    return call,put


In [17]:
pd.DataFrame(data = [{"prices":prices,"call":call,"put":put} for prices,call,put in runMonteCarlo(1,10,ourData,europeanExerciseFunction)])

,prices,call,put
0,[79.7817913020479],12.281791,-12.281791
1,[71.01742266519145],3.517423,-3.517423
2,[68.8943382115733],1.394338,-1.394338
3,[55.25600550527832],-12.243994,12.243994
4,[66.00036748775095],-1.499633,1.499633
5,[57.97414920619942],-9.525851,9.525851
6,[83.47087974297303],15.970880,-15.970880
7,[64.04829440358536],-3.451706,3.451706
8,[81.19915450257444],13.699155,-13.699155
9,[56.58532335844996],-10.914677,10.914677
